## A ANN for classification

In [27]:
import torch
import torch.nn as nn
import numpy as np
from sklearn.cluster import KMeans

In [28]:
# Define the autoencoder model
class Autoencoder(nn.Module):
    def __init__(self, input_dim, encoding_dim):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Linear(input_dim, encoding_dim)
        self.decoder = nn.Linear(encoding_dim, input_dim)

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return encoded, decoded

In [29]:
# generate an array of 20 with random integer values between 0 and 100
x = np.random.randint(0, 1000, 50)

In [30]:
x

array([742, 587, 627,  83, 501, 349, 840, 679, 619, 451, 475, 278, 908,
       491, 518, 216, 837, 332, 606, 220, 161, 706, 239,  97, 542, 477,
       525, 815, 692, 329, 617,  67, 633,  46,  13, 138, 382,  10, 738,
       416, 433, 804, 974,  39, 217, 844, 146, 960, 451, 994])

In [31]:
# Convert the array to a PyTorch tensor
tensor_values = torch.tensor(x, dtype=torch.float32).view(-1, 1)

In [32]:
# Define the dimensions of the autoencoder
input_dim = 1
encoding_dim = 2

In [33]:
# Train the autoencoder
autoencoder = Autoencoder(input_dim, encoding_dim)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(autoencoder.parameters(), lr=0.001, weight_decay=0.0001)

In [34]:
num_epochs = 100
for epoch in range(num_epochs):
    encoded, decoded = autoencoder(tensor_values)
    loss = criterion(decoded, tensor_values)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

In [35]:
# Get the encoded representations for all values
encoded_values, _ = autoencoder(tensor_values)

In [36]:
encoded_values

tensor([[-5.5808e+02,  8.2336e+01],
        [-4.4159e+02,  6.4955e+01],
        [-4.7165e+02,  6.9440e+01],
        [-6.2813e+01,  8.4374e+00],
        [-3.7696e+02,  5.5311e+01],
        [-2.6272e+02,  3.8266e+01],
        [-6.3173e+02,  9.3325e+01],
        [-5.1073e+02,  7.5271e+01],
        [-4.6564e+02,  6.8543e+01],
        [-3.3938e+02,  4.9704e+01],
        [-3.5742e+02,  5.2395e+01],
        [-2.0936e+02,  3.0304e+01],
        [-6.8283e+02,  1.0095e+02],
        [-3.6944e+02,  5.4189e+01],
        [-3.8973e+02,  5.7217e+01],
        [-1.6277e+02,  2.3352e+01],
        [-6.2947e+02,  9.2989e+01],
        [-2.4995e+02,  3.6360e+01],
        [-4.5587e+02,  6.7085e+01],
        [-1.6577e+02,  2.3800e+01],
        [-1.2143e+02,  1.7184e+01],
        [-5.3102e+02,  7.8299e+01],
        [-1.8005e+02,  2.5931e+01],
        [-7.3334e+01,  1.0007e+01],
        [-4.0777e+02,  5.9908e+01],
        [-3.5892e+02,  5.2619e+01],
        [-3.9499e+02,  5.8002e+01],
        [-6.1294e+02,  9.052

In [37]:
# Determine the optimal number of groups using the elbow method
distortions = []
max_clusters = len(x)
for k in range(1, max_clusters + 1):
    kmeans = KMeans(n_clusters=k)
    kmeans.fit(encoded_values.detach().numpy())
    distortions.append(kmeans.inertia_)

c:\Users\erik_\miniconda3\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
c:\Users\erik_\miniconda3\lib\site-packages\sklearn\cluster\_kmeans.py:1382: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
c:\Users\erik_\miniconda3\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
c:\Users\erik_\miniconda3\lib\site-packages\sklearn\cluster\_kmeans.py:1382: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the envir

In [38]:
# Find the optimal number of clusters using the elbow point in the distortion plot
elbow_point = 0
for i in range(1, max_clusters - 1):
    prev_slope = distortions[i] - distortions[i - 1]
    next_slope = distortions[i + 1] - distortions[i]
    if prev_slope > next_slope:
        elbow_point = i + 1
        break

In [39]:
# Group the values using K-means with the optimal number of clusters
kmeans = KMeans(n_clusters=elbow_point)
kmeans.fit(encoded_values.detach().numpy())
cluster_labels = kmeans.labels_

c:\Users\erik_\miniconda3\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
c:\Users\erik_\miniconda3\lib\site-packages\sklearn\cluster\_kmeans.py:1382: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


In [40]:
elbow_point

15

In [41]:
# Print the grouped values
for i in range(elbow_point):
    cluster = [x[j] for j, label in enumerate(cluster_labels) if label == i]
    print(f"Cluster {i + 1}: {cluster}")

Cluster 1: [587, 627, 619, 606, 617, 633]
Cluster 2: [216, 220, 239, 217]
Cluster 3: [974, 960, 994]
Cluster 4: [501, 475, 491, 477]
Cluster 5: [46, 13, 10, 39]
Cluster 6: [742, 738]
Cluster 7: [349, 332, 329, 382]
Cluster 8: [840, 837, 815, 804, 844]
Cluster 9: [451, 416, 433, 451]
Cluster 10: [161, 138, 146]
Cluster 11: [679, 706, 692]
Cluster 12: [518, 542, 525]
Cluster 13: [908]
Cluster 14: [83, 97, 67]
Cluster 15: [278]
